In [13]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from experiments.SL_bound import *
from experiments.DA_bound import *
from bounds.bounds import *
from util.kl import *
from util.misc import *
from results.plotting import *

# Hyper-parameters
TASK = 2

results_folder = "/cephyr/users/frejohk/Alvis/projects/mnist_transfer/results/task2/Binary"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
r_files = [f for f in os.listdir(results_folder) if 'results.pkl' in f]

D = pd.DataFrame({})
for r in r_files: 
    Dr = pd.read_pickle(os.path.join(results_folder, r))
    D = pd.concat([D, Dr], axis=0)

In [17]:
D

,Weightupdates,train_germain,target_germain,KL,e_s,e_t,d_tx,d_sx,error_std,target_error_std,e_s_std,e_t_std,d_tx_std,d_sx_std,alpha,sigma,epsilon,checkpoint,seed
0,0,0.124673,0.216249,0.0,0.062785,0.139770,0.152958,0.123776,0.028561,0.033805,0.004158,0.002307,0.018581,0.014633,0.1,0.030,0.01,1_0,None
0,0,0.077168,0.174432,0.0,0.073378,0.168303,0.012259,0.007578,0.001432,0.003182,0.001008,0.002893,0.001557,0.000786,0.1,0.003,0.01,1_0,None
